In this article, we’ll show you how to use virtual environments to create and manage separate environments for your Python projects, each using different versions of Python for execution. We’ll also take a look at how Python dependencies are stored and resolved.


# Why the Need for Virtual Environments?

Python, like most other modern programming languages, has its own unique way of downloading, storing, and resolving packages (or modules). While this has its advantages, there were some interesting decisions made about package storage and resolution, which has lead to some problems—particularly with how and where packages are stored.

There are a few different locations where these packages can be installed on your system. For example, most system packages are stored in a child directory of the path stored in sys.prefix.

On Mac OS X, you can easily find where sys.prefix points to using the Python shell:

```console
>>> import sys
>>> sys.prefix
'/System/Library/Frameworks/Python.framework/Versions/3.5'
```

More relevant to the topic of this article, third party packages installed using **easy_install** or **pip** are typically placed in one of the directories pointed to by site.getsitepackages:

```
>>> import site
>>> site.getsitepackages()
[
  '/System/Library/Frameworks/Python.framework/Versions/3.5/Extras/lib/python',
  '/Library/Python/3.5/site-packages'
]
```

So, why do all of these little details matter?

It’s important to know this because, by default, every project on your system will use these same directories to store and retrieve site packages (third party libraries). At first glance, this may not seem like a big deal, and it isn’t really, for system packages (packages that are part of the standard Python library), but it does matter for site packages.

Consider the following scenario where you have two projects: ProjectA and ProjectB, both of which have a dependency on the same library, ProjectC. The problem becomes apparent when we start requiring different versions of ProjectC. Maybe ProjectA needs v1.0.0, while ProjectB requires the newer v2.0.0, for example.

This is a real problem for Python since it can’t differentiate between versions in the site-packages directory. So both v1.0.0 and v2.0.0 would reside in the same directory with the same name:

/System/Library/Frameworks/Python.framework/Versions/3.5/Extras/lib/python/ProjectC

Since projects are stored according to just their name, there is no differentiation between versions. Thus, both projects, ProjectA and ProjectB, would be required to use the same version, which is unacceptable in many cases.

This is where virtual environments and the virtualenv/venv tools come into play…

# What Is a Virtual Environment?

At its core, the main purpose of Python virtual environments is to create an isolated environment for Python projects and it physically sits inside a fiolder that contains all of the packages and dependencies that a python project would need. This means that each project can have its own dependencies, regardless of what dependencies every other project has.

In our little example above, we’d just need to create a separate virtual environment for both ProjectA and ProjectB, and we’d be good to go. Each environment, in turn, would be able to depend on whatever version of ProjectC they choose, independent of the other.

The great thing about this is that there are no limits to the number of environments you can have since they’re just directories containing a few scripts. Plus, they’re easily created using the virtualenv or pyenv command line tools.

# Using Virtual Environments

First of all take a look at where the global environment curently lives. We can do that by using the which command:

```console
$ which python
/usr/bin/python
$ which pip
/Library/Frameworks/Python.framework/Versions/3.7/bin/pip

$ which python3
/Library/Frameworks/Python.framework/Versions/3.7/bin/python3
$ which pip3
/Library/Frameworks/Python.framework/Versions/3.7/bin/pip3
```
>  **Note:** If you install it via homebrew on MacOS, it typically resides in /usr/local/bin/pip3

To get started, if you’re not using Python 3, you’ll want to install the virtualenv tool with pip:

**Python 2:**

```console
$ pip install virtualenv
```

**Python 3:**

If you are using Python 3, then you should already have the venv module from the standard library installed.

> **Note:** From here on out, we’ll assume you’re using the newer venv tool, since there are few differences between it and virtualenv with regard to the actual commands. In reality, though, they are very different tools.

Start by making a new directory to work with:

```console
$ mkdir python-virtual-environments && cd python-virtual-environments
```

Create a new virtual environment inside the directory:
```console
# Python 2:
$ virtualenv venv

# Python 3
$ python3 -m venv ./venv
```

> **Note:** By default, this will not include any of your existing site packages.

The Python 3 venv approach has the benefit of forcing you to choose a specific version of the Python 3 interpreter that should be used to create the virtual environment. This avoids any confusion as to which Python installation the new environment is based on.

From Python 3.3 to 3.4, the recommended way to create a virtual environment was to use the pyvenv command-line tool that also comes included with your Python 3 installation by default. But on 3.6 and above, python3 -m venv is the way to go.

In the above example, this command creates a directory called venv, which contains a directory structure similar to this:

> **Note:** tree is an ubuntu command. For MacOS, run: brew install tree

```
$tree venv

├── bin
│   ├── activate
│   ├── activate.csh
│   ├── activate.fish
│   ├── easy_install
│   ├── easy_install-3.5
│   ├── pip
│   ├── pip3
│   ├── pip3.5
│   ├── python -> python3.5
│   ├── python3 -> python3.5
│   └── python3.5 -> /Library/Frameworks/Python.framework/Versions/3.5/bin/python3.5
├── include
├── lib
│   └── python3.5
│       └── site-packages   (This is where all our python third-party packages live)
│           └── ... 
└── pyvenv.cfg
```

Here’s what each folder contains:

* **bin:** files that interact with the virtual environment
* **include:** C headers that compile the Python packages
* **lib:** a copy of the Python version along with a site-packages folder where each dependency is installed

Further, there are copies of, or symlinks to, a few different Python tools as well as to the Python executables themselves. These files are used to ensure that all Python code and commands are executed within the context of the current environment, which is how the isolation from the global environment is achieved. We’ll explain this in more detail in the next section.

More interesting are the activate scripts in the bin directory. These scripts are used to set up your shell to use the environment’s Python executable and its site-packages by default.

In order to use this environment’s packages/resources in isolation, you need to “activate” it. To do this, just run the following:

```console
$ source venv/bin/activate
(venv) $
```

Notice how your prompt is now prefixed with the name of your environment (venv, in our case). This is the indicator that venv is currently active, which means the python executable will only use this environment’s packages and settings.

Now if we run
```console
(venv)saquib$ which python3
/Users/saquib/cloudSync/GoogleDrive/dev/python/venv/bin/python3
(venv)saquib$ which pip3   
/Users/saquib/cloudSync/GoogleDrive/dev/python/venv/bin/pip3
```

It actually points to a different path, within our virtual environment. This means if we install something now, using pip3, it will install it in our virtual environment. Also, when we run

```console
(venv)saquib$ pip3 list
Package    Version
---------- -------
pip        19.0.3 
setuptools 40.8.0 
```

It shows us the basic libraries in our environment which proves its a brand new python environment.
We can install a new library in this virtual environment:
```console
(venv)saquib$ pip install schedule
Collecting schedule
  Downloading schedule-0.6.0-py2.py3-none-any.whl (8.7 kB)
Installing collected packages: schedule
Successfully installed schedule-0.6.0

(venv)saquib$ pip list
Package    Version
---------- -------
pip        20.0.2 
schedule   0.6.0  
setuptools 40.8.0 

```
This gets installed only within this virtual environment and not globally. 

We can to go back to the “system” context by executing deactivate:

```console
(venv) $ deactivate
$
```

Now your shell session is back to normal, and the python command refers to the global Python install. Remember to do this whenever you’re done using a specific virtual environment.

# How Does a Virtual Environment Work?

What exactly does it mean to “activate” an environment? Knowing what’s going on under the hood can be pretty important for a developer, especially when you need to understand execution environments, dependency resolution, and so on.

To explain how this works, let’s first check out the locations of the different python executables. With the environment “deactivated,” run the following:

```console
$ which python
/usr/bin/python
```

Now, activate it and run the command again:

```console
$ source venv/bin/activate
(venv) $ which python
/Users/michaelherman/python-virtual-environments/venv/bin/python
```

After activating the environment, we’re now getting a different path for the python executable because, in an active environment, the $PATH environment variable is slightly modified.

Notice the difference between the first path in $PATH before and after the activation:

```console
$ echo $PATH
/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:

$ source venv/bin/activate
(venv) $ echo $PATH
/Users/michaelherman/python-virtual-environments/venv/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:
```

In the latter example, our virtual environment’s bin directory is now at the beginning of the path. That means it’s the first directory searched when running an executable on the command line. Thus, the shell uses our virtual environment’s instance of Python instead of the system-wide version.

>**Note:** Other packages that bundle Python, like Anaconda, also tend to manipulate your path when you activate them. Just be aware of this in case you run into problems with your other environments. This can become a problem if you start activating multiple environments at once.

This raises the following questions:
* What’s the difference between these two executables anyway?
* How is the virtual environment’s Python executable able to use something other than the system’s site-packages?

This can be explained by how Python starts up and where it is located on the system. There actually isn’t any difference between these two Python executables. **It’s their directory locations that matter.**

When Python is starting up, it looks at the path of its binary. In a virtual environment, it is actually just a copy of, or symlink to, your system’s Python binary. It then sets the location of sys.prefix and sys.exec_prefix based on this location, omitting the bin portion of the path.

The path located in sys.prefix is then used for locating the site-packages directory by searching the relative path lib/pythonX.X/site-packages/, where X.X is the version of Python you’re using.

In our example, the binary is located at 
/Users/michaelherman/python-virtual-environments/venv/bin, 
which means 
sys.prefix would be /Users/michaelherman/python-virtual-environments/venv, 
and therefore the site-packages directory used would be 
/Users/michaelherman/python-virtual-environments/venv/lib/pythonX.X/site-packages. 
Finally, this path is stored in the sys.path array, which contains all of the locations where a package can reside.

# Managing Virtual Environments With virtualenvwrapper
While virtual environments certainly solve some big problems with package management, they’re not perfect. After creating a few environments, you’ll start to see that they create some problems of their own, most of which revolve around managing the environments themselves. To help with this, the **virtualenvwrapper** tool was created. It’s just some wrapper scripts around the main virtualenv tool.

A few of the more useful features of virtualenvwrapper are that it:

* Organizes all of your virtual environments in one location
* Provides methods to help you easily create, delete, and copy environments
* Provides a single command to switch between environments

While some of these features may seem small or insignificant, you’ll soon learn that they’re important tools to add to your workflow.

To get started, you can download the wrapper with pip:
```console
$ pip install virtualenvwrapper
```

> **Note:** For Windows, you should use virtualenvwrapper-win instead.

Once it’s installed, we’ll need to activate its shell functions. We can do this by running source on the installed *virtualenvwrapper.sh* script. When you first install it with pip, the output of the installation will tell you the exact location of virtualenvwrapper.sh. Or you can simply run the following:

```console
$ which virtualenvwrapper.sh
/usr/local/bin/virtualenvwrapper.sh
```

Using that path, add the following three lines to your shell’s startup file. If you’re using the Bash shell, you would place these lines in either the *~/.bashrc* file or the *~/.profile* file. For other shells, like zsh, csh, or fish, you would need to use the startup files specific to that shell. All that matters is that these commands are executed when you log in or open a new shell:

```console
export WORKON_HOME=$HOME/.virtualenvs   # Optional
export PROJECT_HOME=$HOME/projects      # Optional
source /usr/local/bin/virtualenvwrapper.sh
```
> **Note:** It’s not required to define the WORKON_HOME and PROJECT_HOME environment variables. virtualenvwrapper has default values for those, but you can override them by defining values.

Finally, reload the startup file:
```console
$ source ~/.bashrc
```

There should now be a directory located at $WORKON_HOME that contains all of the virtualenvwrapper data/files:

```console
$ echo $WORKON_HOME
/Users/michaelherman/.virtualenvs
```

You’ll also now have the shell commands available to you to help you manage the environments. Here are just a few of the ones available:

* [workon](http://virtualenvwrapper.readthedocs.org/en/latest/command_ref.html#workon)
* [deactivate](http://virtualenvwrapper.readthedocs.org/en/latest/command_ref.html#deactivate)
* [mkvirtualenv](http://virtualenvwrapper.readthedocs.org/en/latest/command_ref.html#mkvirtualenv)
* [cdvirtualenv](https://virtualenvwrapper.readthedocs.org/en/latest/command_ref.html#cdvirtualenv)
* [rmvirtualenv](https://virtualenvwrapper.readthedocs.org/en/latest/command_ref.html#rmvirtualenv)

For more info on commands, installation, and configuring virtualenvwrapper, check out the [documentation](https://virtualenvwrapper.readthedocs.io/en/latest/install.html).

Now, anytime you want to start a new project, you just have to do this:

```console
$ mkvirtualenv my-new-project
(my-new-project) $
```

This will create and activate a new environment in the directory located at $WORKON_HOME, where all virtualenvwrapper environments are stored.

To stop using that environment, you just need to deactivate it like before:

```console
(my-new-project) $ deactivate
$
```

If you have many environments to choose from, you can list them all with the workon function:

```console
$ workon
my-new-project
my-django-project
web-scraper
```

Finally, here’s how to activate:

```console
$ workon web-scraper
(web-scraper) $
```

# Using Different Versions of Python

**Python 2: **

If you would like to be able to use a single tool and switch between Python versions, virtualenv will allow you to do just that. virtualenv has a parameter -p that allows you to select which version of Python to use. Combine that with the which command, and we can easily select your preferred version of Python to use in a simple manner. For example, let’s say that we want Python 3 as our preferred version:

```console
$ virtualenv -p $(which python3) blog_virtualenv
```

This will create a new Python 3 environment.

How does this work? The which command is used for finding a given command in your \$PATH variable and returning the full path to that command. So, the full path to python3 was returned, to the -p parameter which takes a PYTHON_EXE. This could also be used for python2 as well. Just substitute python3 for python2 (or python if you system defaults to python2).

Now you don’t have to remember where you installed your environments. You can easily delete or copy them as you wish, and your project directory is less cluttered!

**python3: **

Unlike the old virtualenv tool, pyvenv doesn’t support creating environments with arbitrary versions of Python, which means you’re stuck using the default Python 3 installation for all of the environments you create. While you can upgrade an environment to the latest system version of Python (via the --upgrade option), if it changes, you still can’t actually specify a particular version.

There are quite a few ways to install Python, but few of them are easy enough or flexible enough to frequently uninstall and re-install different versions of the binary.

This is where [pyenv](https://github.com/yyuu/pyenv) comes in to play.

Despite the similarity in names (pyvenv vs pyenv), pyenv is different in that its focus is to help you switch between Python versions on a system-level as well as a project-level. While the purpose of pyvenv is to separate out modules, the purpose of pyenv is to separate Python versions.

You can start by installing pyenv with either Homebrew (on OS X) or the pyenv-installer project:

**Homebrew:**
```console
$ brew install pyenv
```

**pyenv-installer**
```console
$ curl -L https://raw.githubusercontent.com/yyuu/pyenv-installer/master/bin/pyenv-installer | bash
```
> **Note:** Unfortunately, pyenv does not support Windows. A few alternatives to try are pywin and anyenv.

Once you have pyenv on your system, here are a few of the basic commands you’re probably interested in:
```console
$ pyenv install 3.5.0   # Install new version
$ pyenv versions        # List installed versions
$ pyenv exec python -V  # Execute 'python -V' using pyenv version
```
In these few lines, we install the 3.5.0 version of Python, ask pyenv to show us all of the versions available to us, and then execute the python -V command using the pyenv-specified version.

To give you even more control, you can then use any of the available versions for either “global” use or “local” use. Using pyenv with the local command sets the Python version for a specific project or directory by storing the version number in a local .python-version file. We can set the “local” version like this:
```console
$ pyenv local 2.7.11
```
This creates the .python-version file in our current directory, as you can see here:
```console
$ ls -la
total 16
drwxr-xr-x  4 michaelherman  staff  136 Feb 22 10:57 .
drwxr-xr-x  9 michaelherman  staff  306 Jan 27 20:55 ..
-rw-r--r--  1 michaelherman  staff    7 Feb 22 10:57 .python-version
-rw-r--r--  1 michaelherman  staff   52 Jan 28 17:20 main.py
```
This file only contains the contents “2.7.11”. Now, when you execute a script using pyenv, it’ll load this file and use the specified version, assuming it’s valid and exists on your system.

Moving on with our example, let’s say we have a simple script called main.py in our project directory that looks like this:

In [2]:
import sys
print('Using version:', sys.version[:5])

Using version: 3.7.4


All it does is print out the version number of the Python executable being used. Using pyenv and the exec command, we can run the script with any of the different versions of Python we have installed.
```console
$ python main.py
Using version: 2.7.5
$ pyenv global 3.5.0
$ pyenv exec python main.py
Using version: 3.5.0
$ pyenv local 2.7.11
$ pyenv exec python main.py
Using version: 2.7.11
```

Notice how pyenv exec python main.py uses our “global” Python version by default, but then it uses the “local” version after one is set for the current directory.

This can be very powerful for developers who have lots of projects with varying version requirements. Not only can you easily change the default version for all projects (via global), but you can also override it to specify special cases.

# Conclusion
In this article, you learned about not only how Python dependencies are stored and resolved, but also how you can use different community tools to help get around various packaging and versioning problems.

As you can see, thanks to the huge Python community, there are quite a few tools at your disposal to help with these common problems. As you progress as a developer, be sure to take time to learn how to use these tools to your advantage. You may even find unintended uses for them or learn to apply similar concepts to other languages you use.